In [95]:
import numpy as  np
import matplotlib.pyplot as plt


plt.rcParams['figure.figsize'] = (8.0, 8.0)
plt.rcParams.update({'font.size': 12})

import warnings; warnings.simplefilter('ignore')

from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
#from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.metrics import classification_report, confusion_matrix

import mne
from mne.preprocessing import Xdawn
from mne.decoding import Vectorizer
from mne.viz import tight_layout

In [75]:
filename="/Volumes/flashDrive/data/Ex10_Suj1_Run1-epo.fif"
epoch1 = mne.read_epochs(filename)

Reading /Volumes/flashDrive/data/Ex10_Suj1_Run1-epo.fif ...
Isotrak not found
    Read a total of 1 projection items:
        Average EEG reference (1 x 61) active
    Found the data of interest:
        t =       0.00 ...    1400.00 ms
        0 CTF compensation matrices available
280 matching events found
Created an SSP operator (subspace dimension = 1)
280 matching events found
Created an SSP operator (subspace dimension = 1)
1 projection items activated


In [97]:
#epoch1.plot(n_channels=10,n_epochs=5)

In [77]:
epoch1.event_id

{u'after': 20, u'before': 10, u'new': 4, u'scramble': 5}

In [98]:
#epoch1.events

In [99]:
#epoch1.get_data

In [100]:
#epoch1.info

In [101]:
#epoch1.proj

In [82]:
epoch1.event_id

{u'after': 20, u'before': 10, u'new': 4, u'scramble': 5}

In [102]:
clf = make_pipeline(Xdawn(n_components=3),Vectorizer(),MinMaxScaler(), LogisticRegression(penalty='l1'))

In [103]:
epoch1.event_id

{u'after': 20, u'before': 10, u'new': 4, u'scramble': 5}

In [104]:
len(epoch1.events[:, -1])


280

In [ ]:
# Get the labels
labels = epoch1.events[:, -1]

In [106]:
# Cross validator
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
# Do cross-validation
preds = np.empty(len(labels))
for train, test in cv.split(epoch1, labels):
    clf.fit(epoch1[train], labels[train])
    preds[test] = clf.predict(epoch1[test])


print preds

ValueError: Could not compute eigenvalues, ensure proper regularization (the leading minor of order 61 of 'b' is not positive definite. The factorization of 'b' could not be completed and no eigenvalues or eigenvectors were computed.)

In [71]:
# Classification report
target_names = [u'after', u'before', u'new', u'scramble']
report = classification_report(labels, preds, target_names=target_names)
print(report)

ValueError: Mix type of y not allowed, got types set(['continuous', 'multiclass'])